In [19]:
import pandas as pd
import requests
from tqdm import tqdm
import math

import warnings
warnings.filterwarnings('ignore')

import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [2]:
df_bmf_1608=pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv')
df_bmf_1512=pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv')
df_bmf_1412=pd.read_csv('https://nccs-data.urban.org/data/bmf/2014/bmf.bm1412.csv')
df_bmf_1608['YR_MO']=1608
df_bmf_1512['YR_MO']=1512
df_bmf_1412['YR_MO']=1412

In [3]:
df_bmf_14_16=pd.concat([df_bmf_1608, df_bmf_1412, df_bmf_1512], ignore_index=True)
df_bmf_14_16_confA=df_bmf_14_16[df_bmf_14_16.nteeConf=='A'] # Select A confidence level records.

In [4]:
df_bmf_14_16_confA.sample(5)

,EIN,FIPS,NTEECC,FILER,ZFILER,NAME,ADDRESS,CITY,STATE,ZIP5,...,czFiler,cTaxPer,cAssets,cTotRev,cFinSrc,EPOST,FISYR_IMAGE,IRS990n,NAICS,YR_MO
2219079,363499983,17031.0,T22,Y,N,MCDERMOTT FAMILY FOUNDATION,990 N LAKE SHORE DR APT 31E,CHICAGO,IL,60611,...,N,201212.0,9787.0,1379.0,core2012pf,Z,2014.0,0,813211.0,1412
1957605,260058584,32003.0,S80,Y,Y,KIWANIS INTERNATIONAL INC,PO BOX 29237,LAUGHLIN,NV,89028,...,Y,201309.0,NaN,NaN,submaster990n,Z,2007.0,1,813410.0,1412
3402101,237365075,9003.0,B70,Y,N,SOUTH GLASTONBURY PUBLIC LIBRARY ASSOCIATION INC,PO BOX 169,S GLASTONBURY,CT,6073,...,N,201506.0,361129.0,39787.0,bmf1512c3,Z,2015.0,0,514120.0,1512
513083,310873433,39061.0,J30,Y,N,EASTER SEALS TRISTATE,4300 ROSSPLAIN DR,BLUE ASH,OH,45236,...,N,201412.0,10492751.0,17211403.0,bmf1608c3,Z,2014.0,0,624310.0,1608
3761144,382438763,26147.0,X20,N,N,LIFE IN CHRIST FELLOWSHIP,950 WADHAMS RD,SAINT CLAIR,MI,48079,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813110.0,1512


In [5]:
dview.execute('import pandas as pd')
dview['df_bmf_14_16_confA']=df_bmf_14_16_confA
dview['df_bmf_14_16_confA_ntee_chg']=pd.DataFrame()

In [6]:
# Orgs changed their NTEE codes.
@dview.parallel(block=True)
def ntee_chg(ein):
    global df_bmf_14_16_confA, df_bmf_14_16_confA_ntee_chg
    ntee1_list=df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]['NTEE1']
    if len(set(ntee1_list))!=1:
        df_bmf_14_16_confA_ntee_chg=pd.concat([df_bmf_14_16_confA_ntee_chg, 
                                               df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]]
                                             )

In [7]:
t=ntee_chg.map(df_bmf_14_16_confA.EIN.unique())

In [8]:
df_bmf_14_16_confA_ntee_chg=pd.concat(dview.gather('df_bmf_14_16_confA_ntee_chg'), ignore_index=False)
df_bmf_14_16_confA_ntee_chg[['EIN', 'YR_MO', 'NTEE1']].head(5)

,EIN,YR_MO,NTEE1
65,10063430,1608,C
1584662,10063430,1412,Y
3081824,10063430,1512,Y
113,10132625,1608,U
1584710,10132625,1412,Y


In [9]:
dview['ein_drop_list']=df_bmf_14_16_confA_ntee_chg.EIN.unique()
dview['df_bmf_14_16_confA_chg_drop']=pd.DataFrame()

In [10]:
@dview.parallel(block=True)
def func_ntee_chg_drop(ein):
    global df_bmf_14_16_confA_chg_drop, df_bmf_14_16_confA
    if ein not in ein_drop_list:
        df_bmf_14_16_confA_chg_drop=pd.concat([df_bmf_14_16_confA_chg_drop, 
                                               df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]
                                              ])

In [11]:
t=func_ntee_chg_drop.map(df_bmf_14_16_confA.EIN.unique())

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
df_bmf_14_16_confA_chg_drop=pd.concat(dview.gather('df_bmf_14_16_confA_chg_drop')).sort_index()

In [20]:
for index in range(0, len(df_bmf_14_16_confA_chg_drop)+1, math.ceil(len(df_bmf_14_16_confA_chg_drop)/5)):
    df_temp=df_bmf_14_16_confA_chg_drop.loc[index:index+math.ceil(len(df_bmf_14_16_confA_chg_drop)/5)-1]
    df_temp.to_pickle('../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/df_bmf_14_16_confA_chg_drop.pkl.gz'+'-'+str(df_temp.iloc[0].name)+'-'+str(df_temp.iloc[-1].name), compression='gzip')

In [44]:
len() 
len(df_bmf_14_16_confA.EIN.unique())
# ~1.76% Changed their NTEE codes between 2014-2016. Drop these records since we have no idea on exactly when these codes were changed.

(101045, 960315)

In [45]:
len(df_bmf_14_16_confA[['EIN', 'NTEE1']].drop_duplicates()), len(df_bmf_14_16_confA.EIN.unique())
# ~1.76% Changed their NTEE codes between 2014-2016. Drop these records since we have no idea on exactly when these codes were changed.

(977566, 960315)

In [40]:
df_bmf_14_16_confA_rm_chg=df_bmf_14_16_confA.loc[df_bmf_14_16_confA[['EIN', 'NTEE1']].drop_duplicates(keep=False).index]

In [43]:
len(df_bmf_14_16_confA_rm_chg[['EIN', 'NTEE1']].drop_duplicates(keep=False)), len(df_bmf_14_16_confA_rm_chg)

(199801, 214947)

In [14]:
len(df_bmf_14_16[['EIN', 'NTEE1', 'nteeConf']].drop_duplicates()), len(df_bmf_14_16[['EIN', 'NTEE1', 'nteeConf', 'YR_MO']].drop_duplicates())

(1792488, 4633436)